In [1]:
import random
import h5py
import numpy as np
from scipy.spatial import ConvexHull
from scipy.spatial.transform import Rotation as R
import cv2

###############################################################################
# Diagnostic counters (same as sim code)
###############################################################################
diagnostic_info = {
    "is_inFOV":    {"behind_camera": 0, "margin_fail": 0, "angle_fail": 0},
    "meets_constraints": {"dist_fail": 0, "table_fail": 0, "orientation_fail": 0}
}

###############################################################################
# 1) Utility Functions
###############################################################################
def normalize_quaternion(q):
    """Normalize a quaternion, returning [0,0,0,1] if near zero."""
    norm = np.linalg.norm(q)
    if norm < 1e-6:
        return np.array([0.0, 0.0, 0.0, 1.0], dtype=np.float32)
    return (q / norm).astype(np.float32)

def cart2sph(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)
    el = np.arctan2(z, np.sqrt(x**2 + y**2))
    return r, az, el

def sph2cart(r, az, el):
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

###############################################################################
# 2) Field-of-View with diagnostics (copied from sim code)
###############################################################################
def is_inFOV(camera_pose, camera_fov, point_obj,
             camera_forward_axis='z',
             margin_ratio=0.0,
             img_width=640, img_height=480,
             debug=False, debug_print_limit=30):
    if not hasattr(is_inFOV, "_debug_count"):
        is_inFOV._debug_count = 0

    cam_pos = np.array(camera_pose[:3])
    point_obj = np.array(point_obj)
    dist_to_cam = np.linalg.norm(point_obj - cam_pos)
    if dist_to_cam < 1e-4:
        if debug and is_inFOV._debug_count < debug_print_limit:
            print("[FOV debug] Object too close to camera; bypassing FOV check.")
            is_inFOV._debug_count += 1
        return True

    cam_quat = camera_pose[3:7]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    x_cam, y_cam, z_cam = cam_rot.T.dot(point_obj - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)

    # behind-camera check
    if camera_forward_axis == 'x':
        if x_cam <= 0:
            diagnostic_info["is_inFOV"]["behind_camera"] += 1
            return False
    elif camera_forward_axis == 'z':
        if z_cam <= 0:
            diagnostic_info["is_inFOV"]["behind_camera"] += 1
            return False
    elif camera_forward_axis == '-z':
        if z_cam >= 0:
            diagnostic_info["is_inFOV"]["behind_camera"] += 1
            return False
    else:
        raise ValueError(f"Unknown axis {camera_forward_axis}")

    # projection angles
    if camera_forward_axis in ['z', '-z']:
        h_angle = np.arctan2(x_cam, z_cam)
        v_angle = np.arctan2(y_cam, z_cam)
    else:
        h_angle = np.arctan2(y_cam, x_cam)
        v_angle = np.arctan2(z_cam, x_cam)

    img_x = (h_angle + h_fov/2) / h_fov * img_width
    img_y = (v_angle + v_fov/2) / v_fov * img_height
    margin_x = margin_ratio * img_width
    margin_y = margin_ratio * img_height

    if img_x < margin_x or img_x > img_width - margin_x or \
       img_y < margin_y or img_y > img_height - margin_y:
        diagnostic_info["is_inFOV"]["margin_fail"] += 1
        return False

    if abs(h_angle) > h_fov/2 or abs(v_angle) > v_fov/2:
        diagnostic_info["is_inFOV"]["angle_fail"] += 1
        return False

    return True

###############################################################################
# 3) Additional lift-task constraints (same as sim code)
###############################################################################
def meets_additional_constraints(candidate_pose_7d,
                                 object_pos,
                                 table_z=0.0,
                                 min_dist=0.02,
                                 max_dist=0.30,
                                 max_orientation_deg=180,
                                 eef_approach_axis='x',
                                 world_down_axis=[0,0,-1],
                                 debug=False, debug_print_limit=30):
    if not hasattr(meets_additional_constraints, "_debug_count"):
        meets_additional_constraints._debug_count = 0

    pos = np.array(candidate_pose_7d[:3])
    quat = normalize_quaternion(candidate_pose_7d[3:7])

    # distance
    d = np.linalg.norm(pos - object_pos)
    pass_dist = (d >= min_dist) and (d <= max_dist)
    if not pass_dist:
        diagnostic_info["meets_constraints"]["dist_fail"] += 1

    # table clearance
    pass_table = pos[2] >= (table_z + 0.005)
    if not pass_table:
        diagnostic_info["meets_constraints"]["table_fail"] += 1

    # orientation
    eef_rot = R.from_quat(quat)
    axis = eef_rot.apply([1,0,0]) if eef_approach_axis=='x' else eef_rot.apply([0,0,1])
    dot = np.clip(np.dot(axis, world_down_axis), -1.0, 1.0)
    angle = np.degrees(np.arccos(dot))
    pass_angle = (angle <= max_orientation_deg)
    if not pass_angle:
        diagnostic_info["meets_constraints"]["orientation_fail"] += 1

    all_pass = pass_dist and pass_table and pass_angle
    return all_pass

###############################################################################
# 4) Cube sampler
###############################################################################
def make_cube(cx, cy, cz, rotation, side=0.005):
    half = side/2
    corners = np.array([[-half,-half,-half],
                        [-half,-half, half],
                        [-half, half,-half],
                        [-half, half, half],
                        [ half,-half,-half],
                        [ half,-half, half],
                        [ half, half,-half],
                        [ half, half, half]], dtype=np.float32)
    return (rotation.apply(corners) + np.array([cx,cy,cz],dtype=np.float32))

###############################################################################
# 5) Generate candidates (FOV + extra constraints)
###############################################################################
def generate_candidates_for_demo(positions, goal_pose,
                                 num_samples=5,
                                 margin_ratio=0.0,
                                 table_z=0.0,
                                 camera_fov=(80,80),
                                 camera_forward_axis='z',
                                 eef_approach_axis='x',
                                 debug=False):
    n = len(positions)
    deg2 = np.pi/180
    max_deg = 5*deg2
    max_side = 0.01
    lam = 1.5

    goal_p = np.array(goal_pose[:3])
    dataset = []
    total, passed = 0, 0

    # reset diagnostics
    for m in diagnostic_info.values():
        for k in m: m[k]=0

    for i, p in enumerate(positions):
        dx,dy,dz = p - goal_p
        r, az, el = cart2sph(dx,dy,dz)
        frac = np.exp(-lam*(i/(n-1))) if n>1 else 1.0

        for _ in range(num_samples):
            da = frac*max_deg
            az2 = az + random.choice([-1,1])*da
            el2 = el + random.choice([-1,1])*da
            nx,ny,nz = sph2cart(r,az2,el2) + goal_p

            dr = random.choice([-1,1])*da
            dp = random.choice([-1,1])*da
            dyaw= random.choice([-1,1])*da
            rot = R.from_euler('xyz',[dr,dp,dyaw])
            quat = normalize_quaternion(rot.as_quat())

            center = [nx,ny,nz, *quat]
            total+=1
            in_f = is_inFOV(center, camera_fov, goal_p,
                            camera_forward_axis, margin_ratio,
                            debug=debug)
            ok = in_f and meets_additional_constraints(center, goal_p,
                                                       table_z=table_z,
                                                       min_dist=0.001,
                                                       max_dist=0.40,
                                                       max_orientation_deg=120,
                                                       eef_approach_axis=eef_approach_axis,
                                                       debug=debug)
            if ok:
                passed+=1
                dataset.append({"trajectory_idx":i,"pose_7d":center,"in_fov":True})

            # cube corners
            for corner in make_cube(nx,ny,nz,rot,side=frac*max_side):
                total+=1
                cand = [*corner, *quat]
                inf2 = is_inFOV(cand, camera_fov, goal_p,
                                camera_forward_axis, margin_ratio,
                                debug=debug)
                ok2 = inf2 and meets_additional_constraints(cand, goal_p,
                                                            table_z=table_z,
                                                            min_dist=0.001,
                                                            max_dist=0.40,
                                                            max_orientation_deg=120,
                                                            eef_approach_axis=eef_approach_axis,
                                                            debug=debug)
                if ok2:
                    passed+=1
                    dataset.append({"trajectory_idx":i,"pose_7d":cand,"in_fov":True})

    print(f"Generated {total} candidates; {passed} passed all checks.")
    if passed==0:
        print("=== Diagnostics ===")
        print("FOV:", diagnostic_info["is_inFOV"])
        print("Constraints:", diagnostic_info["meets_constraints"])
    return dataset

###############################################################################
# 6) Compute global goal (copied)
###############################################################################
def compute_global_goal_pose(goal_pose_list):
    pos = np.array([g[:3] for g in goal_pose_list])
    quats = np.array([g[3:7] for g in goal_pose_list])
    mean_p = pos.mean(axis=0)
    mean_q = normalize_quaternion(quats.mean(axis=0))
    return [*mean_p, *mean_q]

###############################################################################
# 7) Convex hull + orientation bounds (same)
###############################################################################
def compute_convex_hull_all(all_candidates, all_main_states):
    pos_list, fwd_list, rotv_list = [], [], []
    for vp in all_candidates:
        p7 = vp["pose_7d"]
        pos_list.append(p7[:3])
        q = normalize_quaternion(p7[3:7])
        fwd_list.append(R.from_quat(q).apply([1,0,0]))
        rotv_list.append(R.from_quat(q).as_rotvec())

    main_p, main_f, main_r = [], [], []
    for states in all_main_states:
        states[:,3:7] = np.array([normalize_quaternion(q) for q in states[:,3:7]])
        main_p.append(states[:,:3])
        main_f.append([R.from_quat(q).apply([1,0,0]) for q in states[:,3:7]])
        main_r.append([R.from_quat(q).as_rotvec()       for q in states[:,3:7]])

    all_p = np.vstack((*main_p, *pos_list))
    all_f = np.vstack((*main_f, *fwd_list))
    all_r = np.vstack((*main_r, *rotv_list))

    hull = ConvexHull(all_p, qhull_options="QJ")
    return all_p, hull, all_r.min(0), all_r.max(0), all_f

###############################################################################
# 8) Save + test safe set (same)
###############################################################################
def save_and_test_safe_set(all_positions_3d, hull, ori_min, ori_max, all_forwards, num_samples=20):
    hull_equ = hull.equations.copy()
    hull_equ[:,-1] -= 0.01  # margin
    np.savez("safe_set_6d_real_robot_additionals_3p.npz",
             safe_set_positions=all_positions_3d,
             hull_equations=hull_equ,
             hull_vertices=hull.vertices,
             orientation_min=ori_min,
             orientation_max=ori_max,
             safe_set_orientations=all_forwards)
    print("Safe set saved.")
    mins, maxs = all_positions_3d.min(0), all_positions_3d.max(0)
    for i in range(num_samples):
        sp = np.random.uniform(mins, maxs)
        rv = np.random.uniform(ori_min, ori_max)
        q  = R.from_rotvec(rv).as_quat()
        inside = np.all(hull_equ[:,:-1].dot(sp) + hull_equ[:,-1] <= 1e-1)
        print(f"Sample {i+1} is {'inside' if inside else 'outside'}.")

###############################################################################
# 9) Raw (16→7) converter (as before)
###############################################################################
def convert_rawstates_16_to_7d(raw_states):
    T = raw_states.shape[0]
    out = np.zeros((T,7),dtype=np.float32)
    for t in range(T):
        row = raw_states[t]
        txyz = row[12:15]
        Rmat = row[0:9].reshape(3,3)
        quat = R.from_matrix(Rmat).as_quat()
        out[t,:3] = txyz
        out[t,3:] = quat
    return out

###############################################################################
# 10) Main for real-robot
###############################################################################
def main_execution_real_robot():
    file_path = "/home/franka_deoxys/data_franka/imgsd_demo/demo.hdf5"
    all_states = []
    goal_list = []
    candidates = []

    with h5py.File(file_path,"r") as f:
        for demo in f["data"].keys():
            grp = f["data"][demo]["obs"]
            if "ee_states" not in grp: continue
            raw = grp["ee_states"][:]
            eef7 = convert_rawstates_16_to_7d(raw)
            all_states.append(eef7)
            if "gripper_states" in grp:
                ci = np.where(grp["gripper_states"][:] < 0.02)[0]
                idx = ci[0] if ci.size>0 else -1
            else:
                idx = -1
            goal_list.append(eef7[idx])

    if not goal_list:
        print("No goals; exiting.")
        return

    global_goal = compute_global_goal_pose(goal_list)

    for states in all_states:
        pos = states[:,:3]
        cps = generate_candidates_for_demo(pos, global_goal,
                                           num_samples=10,
                                           margin_ratio=0.0,
                                           table_z=0.0,
                                           camera_fov=(100,100),
                                           camera_forward_axis='x',
                                           eef_approach_axis='x')
        print(f"Demo had {len(cps)} candidates in FOV & constraints.")
        candidates.extend(cps)

    if not candidates:
        print("No candidates at all; exiting.")
        return

    all_p, hull, omin, omax, fwd = compute_convex_hull_all(candidates, all_states)
    save_and_test_safe_set(all_p, hull, omin, omax, fwd)

if __name__=="__main__":
    main_execution_real_robot()


Generated 27090 candidates; 9676 passed all checks.
Demo had 9676 candidates in FOV & constraints.
Generated 35010 candidates; 0 passed all checks.
=== Diagnostics ===
FOV: {'behind_camera': 29829, 'margin_fail': 5181, 'angle_fail': 0}
Constraints: {'dist_fail': 0, 'table_fail': 0, 'orientation_fail': 0}
Demo had 0 candidates in FOV & constraints.
Generated 26010 candidates; 98 passed all checks.
Demo had 98 candidates in FOV & constraints.
Generated 36000 candidates; 15813 passed all checks.
Demo had 15813 candidates in FOV & constraints.
Generated 32130 candidates; 0 passed all checks.
=== Diagnostics ===
FOV: {'behind_camera': 26399, 'margin_fail': 5731, 'angle_fail': 0}
Constraints: {'dist_fail': 0, 'table_fail': 0, 'orientation_fail': 0}
Demo had 0 candidates in FOV & constraints.
Generated 30060 candidates; 0 passed all checks.
=== Diagnostics ===
FOV: {'behind_camera': 19911, 'margin_fail': 10149, 'angle_fail': 0}
Constraints: {'dist_fail': 0, 'table_fail': 0, 'orientation_fail'

In [2]:
import numpy as np
import plotly.graph_objects as go
from scipy.spatial import ConvexHull

def plot_interactive_safe_set_with_cones(
    npz_filepath="safe_set_6d_simulation_seed_10_6.npz",
    max_cones=500,
    cone_scale=0.02
):
    """
    Load safe-set data (positions + forward vectors) from an NPZ file,
    plot the convex hull of positions, and draw orientation cones (subsampled).

    Args:
        npz_filepath (str): Path to the NPZ file containing:
            - "safe_set_positions": (N,3) array of 3D positions
            - "safe_set_orientations": (N,3) array of forward vectors
        max_cones (int): maximum number of cones to render (to avoid freezing)
        cone_scale (float): how large each cone is drawn
    """
    # Load data from the NPZ file
    data = np.load(npz_filepath)
    safe_set_positions = data["safe_set_positions"]  # shape (N, 3)
    safe_set_orientations = data["safe_set_orientations"]  # shape (N, 3)
    
    # Build the convex hull for the positions
    hull = ConvexHull(safe_set_positions, qhull_options="QJ")

    # 1) Scatter trace of the raw positions
    scatter_trace = go.Scatter3d(
        x=safe_set_positions[:, 0],
        y=safe_set_positions[:, 1],
        z=safe_set_positions[:, 2],
        mode="markers",
        marker=dict(size=3, color="blue"),
        name="Safe Set Positions"
    )

    # 2) Mesh trace for the convex hull
    mesh_trace = go.Mesh3d(
        x=safe_set_positions[:, 0],
        y=safe_set_positions[:, 1],
        z=safe_set_positions[:, 2],
        i=hull.simplices[:, 0],
        j=hull.simplices[:, 1],
        k=hull.simplices[:, 2],
        opacity=0.3,
        color="red",
        name="Convex Hull"
    )

    # 3) Subsample orientation cones
    N = len(safe_set_positions)
    if N > max_cones:
        # random or evenly spaced subsample
        indices = np.linspace(0, N-1, max_cones, dtype=int)
    else:
        indices = np.arange(N)

    subsampled_positions = safe_set_positions[indices]
    subsampled_orientations = safe_set_orientations[indices]

    # Normalize orientation vectors so that we can scale them uniformly
    norms = np.linalg.norm(subsampled_orientations, axis=1, keepdims=True)
    norms[norms == 0] = 1.0  # avoid division by zero
    unit_orientations = subsampled_orientations / norms

    # Adjust the scale of cones as needed
    u = unit_orientations[:, 0] * cone_scale
    v = unit_orientations[:, 1] * cone_scale
    w = unit_orientations[:, 2] * cone_scale

    cone_trace = go.Cone(
        x=subsampled_positions[:, 0],
        y=subsampled_positions[:, 1],
        z=subsampled_positions[:, 2],
        u=u,
        v=v,
        w=w,
        sizemode="absolute",
        sizeref=cone_scale,
        anchor="tail",
        colorscale="Blues",
        showscale=False,  # hide the color scale legend
        name="Orientation Cones",
    )

    # Combine all traces into a single figure
    fig = go.Figure(data=[mesh_trace, scatter_trace, cone_trace])
    fig.update_layout(
        title="Interactive Safe Set with Orientation Cones",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z"
        )
    )
    fig.show()

if __name__ == "__main__":
    file_name = 'safe_set_6d_real_robot_additionals_3p.npz'
    plot_interactive_safe_set_with_cones(file_name)
